In [1]:
f1 = open('review_5k_user', 'r').read()
review_5k_user = eval(f1)

f2 = open('review_5k_business', 'r').read()
review_5k_business = eval(f2)

f3 = open('review_5k_rating', 'r').read()
review_5k_rating = eval(f3)

f4 = open('relation_5k', 'r').read()
relation_5k = eval(f4)


In [23]:
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
import networkx as nx
import matplotlib.pyplot as plt
from time import time

def matrix_factorization(R, P, Q, K, S, H, N, Si, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] * S[i][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] * S[i][j]- beta * Q[k][j])
            #for m in N[i]:
            #    Eim=Si[i][m]-np.sum(P[i,:]*H*P[j,:]);
            #    for k in xrange(K):
            #        P[i][k]=P[i][k]+alpha*(2 * Eim * beta * H[k] * P[m][k]);
            #       H[k]=H[k]+ alpha*(2 * Eim * beta * P[i][k] * P[m][k]);
        eR = np.dot(P,Q)
        e = 0
        E= 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
            for m in N[i]:
                E = E + pow(Si[i][m]-np.sum(P[i,:]*H*P[j,:]),2);
        if e + E/2 < 0.001:
            break
    return P, Q.T, H

In [3]:
num_user = 4929
num_business = 2686
num_review = 49486
Nij=[[]]*num_user;

for r in relation_5k:
    Nij[r[0]]=r[1];

In [4]:
row = np.array([])
col = np.array([])
val = np.array([])

In [5]:
Tij = csr_matrix((val,(row,col)), shape=(num_user,num_user)).toarray()
for u in relation_5k:
    for f in u[1]:
        Tij[u[0]][f] = 1

In [6]:
none_zero=[];
Rij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_review):
    Rij[review_5k_user[r]][review_5k_business[r]] = review_5k_rating[r];
    none_zero.append([review_5k_user[r],review_5k_business[r]]);

In [7]:
Sij_norm = []
for u in xrange(num_user):
    Sij_norm.append(math.sqrt(np.dot(Rij[u], Rij[u])))

In [8]:
Sij = np.dot(Rij, Rij.T)

In [9]:
for i in xrange(num_user):
    for j in xrange(num_user):
        if Sij_norm[i] and Sij_norm[j]:
            Sij[i][j] = Sij[i][j]/Sij_norm[i]/Sij_norm[j]
        else:
            Sij[i][j]=0

In [10]:
print Sij[0][:]

[ 1.          0.          0.         ...,  0.          0.07236504  0.        ]


In [10]:
import networkx as nx

node_list = []
for u in xrange(num_user):
    node_list.append(u)

In [11]:
G=nx.Graph()
G.add_nodes_from(node_list)
for u in relation_5k:
    for f in u[1]:
        G.add_edge(u[0], f)

In [12]:
PR = nx.pagerank(G)

In [13]:
from operator import itemgetter
sorted_PR = sorted(PR.items(), key=itemgetter(1), reverse=True)


In [14]:
rank = {}
for u in xrange(num_user):
    rank[sorted_PR[u][0]] = u

In [15]:
Ri = []
for u in xrange(num_user):
    Ri.append(rank[u]+1)

In [16]:
Wij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        if Rij[i][j] > 0:
            Wij[i][j] = math.sqrt(Ri[i])

In [24]:
def performance(P, Q, R, none_zero):
    num=len(none_zero)
    Q=Q.T;
    r=np.dot(P,Q);
    error=0;
    for p in none_zero:
        i=p[0];
        j=p[1];
        error+=(R[i][j]-r[i][j])*(R[i][j]-r[i][j]);
        #print R[i][j]-r[i][j]
    error=math.sqrt(error/num);
    return error;
    
n=10;
testnum=num_review*0.1;
times=1;
total=len(none_zero);
k=8;
rmse=0.;
steps=10;
start=time();
true_times=0;
for x in xrange(times):
    chose=np.random.randint(0,total,size=testnum);
    not_chose=list(set(range(0,total))-set(chose));
    trainmat = np.copy(Rij)
    train_none_zero=list(none_zero[i] for i in not_chose);
    for p in chose:
        i=none_zero[p][0];
        j=none_zero[p][1];
        trainmat[i][j]=0;
    test_none_zero=list(none_zero[i] for i in chose);
    P = np.random.rand(num_user,k);
    Q = np.random.rand(num_business,k);
    H = np.random.rand(k)
    nP, nQ , nH= matrix_factorization(trainmat, P, Q, k, Wij, H, Nij, Sij, steps, 0.0002, 0.2);
    #nP, nQ = changed_matrix_factorization(ratemat, P, Q, k, train_none_zero, steps, 0.01);
    rmse+=performance(nP,nQ,Rij,test_none_zero);
    true_times+=1;
    if(time()-start>36):
        break;
    
print rmse/true_times;

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1.03513584469


In [73]:
print np.dot(nP,nQ.T)[0][0:100]

[ 5.44624646  6.4309799   6.4426096   7.4459624   7.10042252  6.18539953
  5.94344888  6.43881055  5.86717316  7.55870824  7.65267952  7.11484289
  7.0421867   7.26567287  6.71465661  9.55555544  6.1752896   6.52197278
  5.66845841  6.48178143  4.75684367  5.62385368  5.54143585  7.29503414
  4.74026079  6.66985384  6.80857198  6.53989149  6.19676978  5.24858402
  5.58417099  7.28232536  6.29716209  7.2690556   5.89677317  6.45558601
  6.61861048  6.98097843  7.03129578  5.93956293  5.44406548  6.44154849
  6.90988789  6.19209653  6.93528806  5.78069995  6.5811211   6.45433347
  7.23929709  5.51324454  5.33760163  4.87750364  7.88634849  6.3221905
  6.59827536  7.18643478  5.96857461  7.59698597  5.56955314  5.95928458
  6.10061315  6.7509699   6.18429604  6.60495092  6.99774896  7.59341558
  6.88182675  5.09127157  6.65404848  6.50656668  7.02767555  7.4516433
  6.11832209  7.05960922  5.65732158  5.3612169   5.27338469  7.42949251
  6.55642675  5.70559171  6.10202425  7.5723179   7.1